#Downloading Solar Data with SunPy

This lesson covers the functionality in SunPy to download solar physics data from various places. It has been written for the current release (v0.5.4), despite the fact there are large improvements in the downloading software for SunPy v0.6.

This lesson will cover the core downloaders availible in the `sunpy.net` submodule, we will cover the following topics:

* The Virtual Solar Observatory.
* The SunPy query syntax.
* Downloading data and managing files.

## The Virtual Solar Observatory

The VSO is a centralised portal to many different sources of solar physics data. SunPy has a very powerful and easy to use VSO client, which we are going to use to download an AIA image.

Import the `vso` module:

In [ ]:
from sunpy.net import vso

Create a client instance:

In [ ]:
vclient = vso.VSOClient()

Query the VSO:

In [ ]:
results = vclient.query(vso.attrs.Time('2015/03/15T00:00:00', '2015/03/15T00:00:01') & vso.attrs.Instrument('AIA'))
results.show()

Download the File:

In [ ]:
files = vclient.get(results).wait(progress=True)
files

## SunPy Query Syntax

Having done a quick demo of the VSO client, we will now go through the steps in more detail and explain them. The  main part of the VSO interface is the query syntax.
The query syntax allows you to perform very complex queries, by using the logical and `&` and logical or `|` operators on the indivual query attributes.

The query attributes live in the `sunpy.net.vso.attrs` namespace, which we have imported as `vso.attrs`.

The attributes available for use in a VSO query are:

In [ ]:
vso.attrs.__all__

You can combine these attributes to make simple or more complex queries.
First, let's explain the example we used earlier:

`vclient.query(vso.attrs.Time('2015/03/15T00:00:00', '2015/03/15T00:00:01') & vso.attrs.Instrument('AIA'))`

In [ ]:
vso.attrs.Time('2015/03/15T00:00:00', '2015/03/15T00:00:01')

The `Time` attribute, specifies a time range for the data, you must specify a start and an end value to `Time`.

In [ ]:
vso.attrs.Instrument('AIA')

The `Instrument` attribute specifies the name of the instrument used to collect the data, the list of availible instruments is on the VSO website: http://sdac.virtualsolar.org/cgi/show_details?keyword=INSTRUMENT

When we passed these into `query()` we used the `&` symbol to tell `query()` that we wanted VSO to look for records that satisfied both attributes.

It is also possible to use the `|` operator to get multiple results:

In [ ]:
vclient.query(vso.attrs.Time('2015/03/15T00:00:29', '2015/03/15T00:00:30')
              & (vso.attrs.Instrument('AIA') | vso.attrs.Instrument('HMI'))).show()

We can also construct these queries outside of the `query()` method, to make it easier to read or to make multiple queries:

In [ ]:
time = vso.attrs.Time('2015/03/15T00:00:00', '2015/03/15T00:01:00')
aia = vso.attrs.Instrument('AIA') & vso.attrs.Sample('60')
hmi = vso.attrs.Instrument('HMI') & vso.attrs.Physobs('LOS_magnetic_field')

In [ ]:
vclient.query(time, aia | hmi).show()

<div style='background:#B1E0A8; padding:10px 10px 10px 10px;'>
<h2>Challenges</h2>
<p>
<ol>
<li> Create a query that searches for an AIA image on your last birthday</li>
<li> Create a query that retrieves an image from AIA, HMI and LASCO C2 taken approximately at the same time.</li>
</p>
</div>

### Solution 1

In [ ]:
vclient.query(vso.attrs.Time('2014/05/12T02:00:00', '2014/05/12T02:01:00'),
              vso.attrs.Instrument('AIA'), vso.attrs.Sample('60'),
              vso.attrs.Wave(171, 171)).show()

### Solution 2

In [ ]:
aia = vso.attrs.Instrument('AIA') & vso.attrs.Sample('60')
hmi = vso.attrs.Instrument('HMI') & vso.attrs.Physobs('LOS_magnetic_field')
lasco = vso.attrs.Instrument('LASCO')

vclient.query(vso.attrs.Time('2014/05/12T02:00:00', '2014/05/12T02:01:00'), aia | hmi | lasco).show()

## Retreiving Files

So far we have been downloading files using `get()` with the default options. In this section we are going to look at how to customise these options and save the files to different directories.

In [ ]:
time = vso.attrs.Time('2014/05/12T02:00:00', '2014/05/12T02:01:00')
aia = vso.attrs.Instrument('AIA') & vso.attrs.Sample('60')

In [ ]:
results = vclient.query(time, aia | hmi)
results.show()

Once we have obtained the results we can download them with the `get()` method:

In [ ]:
vclient.get?

The important parameters for the get method are the `path` and the `methods` keyword arguments.

`path` allows you to specify where the data is downloaded to, by default it goes to the SunPy data directory, which defaults to `~/sunpy/data`, this (and other options) can be set in the SunPy config file `~/.sunpy/sunpyrc`.

In [ ]:
vclient.get(results, path='/home/stuart/sunpy/data/{instrument}/{file}.fits').wait()

The parameters that you can use for the path formatting is anything that is in the `QueryResponse` object, or the results from query:

In [ ]:
results[0]

Things that have multiple nested properties can be acessed via `.` i.e. `time.start` or `wave.wavemin`.

`method` allows you to request the data in a different form, such as a tarball or a RICE compressed image for AIA:

In [ ]:
results = vclient.query(time, aia)
vclient.get(results, methods=['URL-FILE_Rice']).wait()

<div style='background:#B1E0A8; padding:10px 10px 10px 10px;'>
<h2>Challenges</h2>
<p>
<ol>
<li>Download an image of your choice and display it using SunPy's Map class.</li>
<li>Download a set of AIA images (RICE compressed) and put them into folders based on their wavelength.</li>
</p>
</div>

### Solution 1

In [ ]:
results = vclient.query(time, aia)
files = vclient.get(results).wait()
files

In [ ]:
import sunpy.map
%matplotlib inline

In [ ]:
aiam = sunpy.map.Map(files)
fig = aiam.peek()

###Solution 2

In [ ]:
time = vso.attrs.Time('2014/05/12T02:00:00', '2014/05/12T02:00:12')
aia = vso.attrs.Instrument('AIA')

results = vclient.query(time, aia)
results.show()

In [ ]:
files = vclient.get(results, methods=['URL-FILE_Rice'], path='/home/stuart/sunpy/data/{wave.wavemin}/{file}').wait()
files